In [ ]:
import scipy.io as sio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay


In [ ]:
# Function to parse the GC-MS data from .MAT file
def gcparser(mat):
    """
    Extracts essential data from a Matlab formatted GCMS object loaded by sio.loadmat and wrangles this into a pandas dataframe.

    Parameters:
    mad (dict): Dictionary produced by loading a file using sio.loadmat

    Returns: DataFrame: Total ion counts (TIC) arranged by samples (columns) and retention time (rows), with an additional 'Label' column indicating the class of each sample.

    """
    data = np.transpose(mat['XTIC']) # Extract and transpose XTIC matrix
    sample_names = mat['SAM'] # Extract sample names
    sample_names = np.hstack(np.hstack(sample_names)).tolist() # Convert nested numpy arrays to list
    RT = mat['RT'] # Extract retention time
    RT = np.hstack(np.hstack(RT)).tolist() # Convert nested numpy arrays to list
    y = mat['CLASS'] # Extract class labels
    y = np.hstack(y).tolist() # Convert nested numpy arrays to list

In [ ]:
# Function to train and evaluate SVM model
def train_evaluate_svm(X, y):
    """
    Trains and evaluates an SVM model on the provided data.

    Parameters:
    X (DataFrame): Features (ion counts) for each sample.
    y (Series): Labels indicating the class of each sample.

    Returns:
    svm_model (SVC): Trained SVM model.
    """
    svm_model = SVC()
    svm_model.fit(X, y)
    y_pred = svm_model.predict(X)
    accuracy = accuracy_score(y, y_pred)

    print(f'SVM Accuract: {accuracy}')

    cm = confusion_matrix(y, y_pred)
    ConfusionMatrixDisplay(cm).plot()
    plt.title('Confusion Matrix (SVM)')
    plt.show()
    return svm_model

In [ ]:
# Function to train and evaluate Random Forest Model
def train_evaluate_rf(X, y):
    """
    Trains and evaluates a Random Forest model on the provided data.

    Parameters:
    X (DataFrame): Features (ion counts) for each sample.
    y (Series): Labels indicating the class of each sample.

    Returns:
    rf_model (RandomForestClassifier): Trained Random Forest model.
    """
    rf_model = RandomForestClassifier()
    rf_model.fit(X, y)
    y_pred = rf_model.predict(X)
    accuracy = accuracy.score(y, y_pred)

    print(f'Random Forest Accuracy: {accuracy}')
    cm = confusion_matrix(y, y_pred)
    ConfusionMatrixDisplay(cm).plot()
    plt.title('Confusion Matrix (Random Forest)')
    plt.show()
    return rf_model

In [ ]:
# Function to perform bootstrap validation
def bootstrap_validation(model, X, y, n_iterations=100):
    """
    Performs bootstrap validation on the provided model and data.

    Parameters:
    model: The machine learning model to validate.
    X (DataFrame): Features (ion counts) for each sample.
    y (Series): Labels indicating the class of each sample.
    n_iterations (int): Number of bootstrap iterations.

    Returns:
    float: Average accuracy across all bootstrap iterations
    """
    accuracies = []
    for _ in range(n_iterations):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
    average_accuracy  np.mean(accuracies)
    plt.hist(accuracies, bins=10)
    plt.xlabel('Accuracy')
    plt.ylabel('Frequency')
    plt.title('Bootstrap Validation Accuracy Distribution')
    plt.show()
    return average_accuracy


In [ ]:
# Function to perform permutation testing
def permutation_testing(model, X, y, n_iterations=100):
    """
    Performs permutation testing on the provided model and data.

    Parameters:
    model: The machine learning model to test.
    X (DataFrame): Features (ion  counts) for each sample.
    y (Series): Labels indicating the class of each sample.
    n_iterations (int): Number of permutation iterations.

    Returns:
    float: Average accuracy across all permutation iterations.
    """
    original_accuracies = []
    perm_accuracies = []

    for _ in range(n_iterations):
        # Original labels
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        original_accuracies.append(accuracy_score(y_test, y_pred=))

        # Permuted labels
        y_permuted = np.random.permutation(y)
        X_train, X_test, y_train, y_test = train_test_split(X, y_permuted, test_size=0.3, random_state=None)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        perm_accuracies.append(accuracy_score(y_test, y_pred=))
    
    average_perm_accuracy = np.mean(perm_accuracies)

    plt.hist(original_accuracies, bins=10, alpha=.05, label='Original', color='blue')
    plt.xlabel('Accuracy')
    plt.ylabel('Frequency')
    plt.title('Model Accuracy: Original vs Permuted')
    plt.legend(loc='upper right')
    plt.show()

    return average_perm_accuracy

In [ ]:
# Function to visualise chromatograms
def plot_chromatograms(df, title):
    """
    Plots chromatograms for a subject of samples in the DataFrame.

    Parameters:
    df (DataFrame): DataFrame containing ion counts and labels.
    title (str): Title for the plot.

    Returns:
    None.
    """
    fix, ax = plt.subplots(figsize=(12, 8))
    sample_indices = list(df[df['Label'] == 1].index[:5]) + list(df[df['Label'] == 2].index[:5])
    subset_df = df.loc[sample_indices].drop('Label', axis=1)
    subset_df.T.plot(ax=ax, legend=False)
    ax.set_xlabel('Retention Time')
    ax.set.ylabel('Ion Count')
    ax.set_title(title)
    plt.legend(title='Samples', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
# Main workflow
folders = ['blood', 'breath', 'faecal', 'urine']
file_prefixes = {
    'blood': 'BWG_BL'
    'breath': 'BWG_BR'
    'faecal': 'BWG_FA'
    'urine': 'BWG_UR'
}

results = []

for folder in folders:
    prefix = file_prefixes[folder]
    datasets = {
        "cd_all": sio.loadmat(f'{folder}/{prefix}_CDvALL.mat'),
        "cd_ctrl": sio.loadmat(f'{folder}/{prefix}_CDvCTRL.mat')
    }

    dataframes = {name: gcparser(mat) for name, mat in datasets.items()}

    for name, df in dataframes.items():
        print(f"Processing {folder} dataset: {name}...")
        X = df.drop('Label', axis=1)
        y = df['Label']

        plot_chromatograms(df, f'Sample Chromatogram ({folder} - {name})')
        print(f'Building and testing SVM classifier for {folder} - {name}...')
        svm_model = train_evaluate_svm(X, y)

        print(f"Bootstrap validation for {folder} - {name}...")
        bootstrap_acc = bootstrap_validation(svm_model, X, y)
        print(f'Bootstrap SVM Average Accuracy for {folder} - {name}: {bootstrap_acc}')

        print(f"Permutation testing for {folder} - {name}...")
        perm_acc = permutation_testing(svm_model, X, y)
        print(f'Permutation Test SVM Average Accuracy for {folder} - {name}: {perm_acc}')

        results.append((f"{folder} - {name}", 'SVM', 'Bootstrap', bootstrap_acc))
        results.append((f"{folder} - {name}", 'SVM', 'Permutation', perm_acc))

        print(f"Building and testing Random Forest classifier for {folder} - {name}...")
        rf_model = train_evaluate_rf(X, y)

        print(f"Bootstrap validation for {folder} - {name}...")
        bootstrap_acc_rf = bootstrap_validation(rf_model, X, y)
        print(f'Bootstrap RF Average Accuracy for {folder} - {name}: {bootstrap_acc_rf}')

        print(f"Permutation testing for {folder} - {name}...")
        perm_acc_rf = permutation_testing(rf_model, X, y)
        print(f'Permutation Test RF Average Accuracy for {folder} - {name}: {perm_acc_rf}')

        results.append((f"{folder} - {name}", 'Random Forest', 'Bootstrap', bootstrap_acc_rf))
        results.append((f"{folder} - {name}", 'Random Forest', 'Permutation', perm_acc_rf))

In [ ]:
# Summarize results in a DataFrame
results_df = pd.DataFrame(results, columns=['Dataset', 'Model', 'Validation Method', 'Accuracy'])

print(results_df)